## Import Libraries

In [1]:
import test_template as tt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Alternative Optimisers

### Generate .MPS file with Gurobi

#### Import Libraries

In [1]:
from scheduler_v2 import Scheduler
from ScheduleSimulator import SchedulerSimulation as SchedSim
filename = "test_model_main.mps"

#### Generate Model File

In [2]:
s = SchedSim("sample_input/sample_input_v2_3.json")
s.process_event(s.events[0])
requests, resources = s.prepare_scheduler()
sched = Scheduler(s.now, s.horizon, s.slice_size, resources, s.proposals, requests, verbose=0)
sched.calculate_free_windows()
sched.build_data_structures()
sched.build_model()
sched.write_model("new_test_file.mps")

#### Solve with Gurobi

In [3]:
sched.load_model(filename)
sched.solve_model()
g_sol = sched.return_solution()

In [3]:
import gurobipy
filename = "enlight_hard.mps"
model = gurobipy.read("new_test_file.mps")
model.optimize()
print(f"Objective: {model.ObjVal}")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-20
Read MPS format model from file new_test_file.mps
Reading time = 0.23 seconds
Test: 711 rows, 21693 columns, 106950 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 711 rows, 21693 columns and 106950 nonzeros
Model fingerprint: 0x2b85e252
Variable types: 0 continuous, 21693 integer (21693 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 22175.989190
Presolve removed 23 rows and 472 columns
Presolve time: 0.40s
Presolved: 688 rows, 21221 columns, 103837 nonzeros
Found heuristic solution: objective 36511.719201
Variable types: 0 continuous, 21221 i

### HiGHS

In [4]:
import highspy
h = highspy.Highs()

In [5]:
status = h.readModel(filename)
print(status)
h.run()

solution = h.getSolution()
info = h.getInfo()
model_status = h.getModelStatus()
print("Model Status = ", h.modelStatusToString(model_status))
print("Optimal Objective = ", info.objective_function_value)

HighsStatus.kOk
Model Status =  Optimal
Optimal Objective =  37.0


In [5]:
cv = solution.col_value

for i in range(len(cv)):
    if cv[i] > 0.5:
        print(sched.yik[i])

[6, 24, 789.6428975928169, 't0', 0, <scheduler_utils.PossibleStart object at 0x0000024B61470910>]
[31, 27, 589.112687224596, 't0', 0, <scheduler_utils.PossibleStart object at 0x0000024B6150E6D0>]
[32, 1, 966.564840781936, 't1', 0, <scheduler_utils.PossibleStart object at 0x0000024B6151CD10>]
[33, 13, 188.5491771372604, 't1', 0, <scheduler_utils.PossibleStart object at 0x0000024B6151EB90>]
[36, 93, 67.6505827194146, 't4', 0, <scheduler_utils.PossibleStart object at 0x0000024B61533F90>]
[55, 11, 1522.4043285722641, 't3', 0, <scheduler_utils.PossibleStart object at 0x0000024B61590AD0>]
[60, 3, 1742.6443059376518, 't3', 0, <scheduler_utils.PossibleStart object at 0x0000024B615B1D50>]
[70, 6, 1978.0724539871, 't0', 0, <scheduler_utils.PossibleStart object at 0x0000024B618156D0>]
[86, 21, 1549.8383807970579, 't3', 0, <scheduler_utils.PossibleStart object at 0x0000024B61872ED0>]
[104, 75, 332.2713207766198, 't1', 0, <scheduler_utils.PossibleStart object at 0x0000024B618D5010>]
[113, 40, 984.0

In [6]:
h_scheduled = []
rv = solution.row_value
for i in range(len(rv)):
    if rv[i] > 0.5:
        const_name = h.getRowName(i)[1]
        if "one_per_reqid_constraint_" in const_name:
            h_scheduled.append(const_name.replace("one_per_reqid_constraint_", ""))
print(h_scheduled)

['6', '31', '32', '33', '36', '55', '60', '70', '86', '104', '113', '114', '117', '141', '143', '153', '167', '170', '203', '208', '213', '250', '264', '270', '301', '302', '314', '317', '323', '361', '375', '398', '414', '416', '425', '432', '439', '467', '470']


In [7]:
list(g_sol["scheduled"].keys()) == h_scheduled

True

### CPL
Coin-OR Linear Problem Solver, run using the Python package CyPL

In [10]:
from cylp.cy import CyClpSimplex
s = CyClpSimplex()
s.readMps(filename)
s.dual()
round(s.objectiveValue, 3)

0.0

In [ ]:
from cylp.cy import CyClpSimplex

In [ ]:
s = CyClpSimplex()

In [ ]:
s.readMps(filename)

In [ ]:
s.primal()

In [ ]:
round(s.objectiveValue, 3)

In [23]:
s.objectiveValue

0.0

In [27]:
s.objective.sum()

9254451.756768674

In [14]:
s.primalVariableSolution.sum()

0.0

In [15]:
import numpy as np
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray

s = CyClpSimplex()

# Add variables
x = s.addVariable('x', 3)
y = s.addVariable('y', 2)

# Create coefficients and bounds
A = np.matrix([[1., 2., 0],[1., 0, 1.]])
B = np.matrix([[1., 0, 0], [0, 0, 1.]])
D = np.matrix([[1., 2.],[0, 1]])
a = CyLPArray([5, 2.5])
b = CyLPArray([4.2, 3])
x_u= CyLPArray([2., 3.5])

# Add constraints
s += A * x <= a
s += 2 <= B * x + D * y <= b
s += y >= 0
s += 1.1 <= x[1:3] <= x_u

# Set the objective function
c = CyLPArray([1., -2., 3.])
s.objective = c * x + 2 * y.sum()

# Solve using primal Simplex
s.primal()
print(s.primalVariableSolution['x'])

[0.2 2.  1.1]


In [16]:
s.objectiveValue

1.3000000000000003

In [17]:
s.objective

array([ 1., -2.,  3.,  2.,  2.])

In [29]:
import numpy as np
from cylp.cy import CyCbcModel, CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPModel, CyLPArray
model = CyLPModel()

x = model.addVariable('x', 3, isInt=True)
y = model.addVariable('y', 2)

A = np.matrix([[1., 2., 0],[1., 0, 1.]])
B = np.matrix([[1., 0, 0], [0, 0, 1.]])
D = np.matrix([[1., 2.],[0, 1]])
a = CyLPArray([5, 2.5])
b = CyLPArray([4.2, 3])
x_u= CyLPArray([2., 3.5])

model += A*x <= a
model += 2 <= B * x + D * y <= b
model += y >= 0
model += 1.1 <= x[1:3] <= x_u

c = CyLPArray([1., -2., 3.])
model.objective = c * x + 2 * y.sum()

s = CyClpSimplex(model)

cbcModel = s.getCbcModel()

cbcModel.solve()

print (cbcModel.status)
sol_x = cbcModel.primalVariableSolution['x']

(abs(sol_x - np.array([0, 2, 2])) <= 10**-6).all()

solution


True

In [31]:
s = CyClpSimplex()
s.readMps(filename)
cbcModel = s.getCbcModel()
cbcModel.solve()
print(cbcModel.status)

solution


In [36]:
sol_x = cbcModel.primalVariableSolution

In [37]:
sol_x

array([0., 0., 0., ..., 0., 0., 0.])

## OUTDATED - Alternative Optimiser - HiGHS

In [2]:
from scheduler_v2 import Scheduler
from ScheduleSimulator import SchedulerSimulation as SchedSim
import highspy
h = highspy.Highs()
filename = "test_model_main.mps"

### Gurobi - Create Model

In [3]:
s = SchedSim("sample_input/sample_input_v2_3.json")
s.process_event(s.events[0])
requests, resources = s.prepare_scheduler()
sched = Scheduler(s.now, s.horizon, s.slice_size, resources, s.proposals, requests, verbose=0)
sched.calculate_free_windows()
sched.build_data_structures()
sched.build_model()

### Gurobi - Solve Model

In [4]:
sched.load_model(filename)
sched.solve_model()
g_sol = sched.return_solution()

21693


In [5]:
print(list(g_sol["scheduled"].keys()))

['6', '31', '32', '33', '36', '55', '60', '70', '86', '104', '113', '114', '117', '141', '143', '153', '167', '170', '203', '208', '213', '250', '264', '270', '301', '302', '314', '317', '323', '361', '375', '398', '414', '416', '425', '432', '439', '467', '470']


### HiGHS - Solve Model

In [6]:
status = h.readModel(filename)
print(status)
h.run()

HighsStatus.kOk


<HighsStatus.kOk: 0>

In [7]:
solution = h.getSolution()
info = h.getInfo()
model_status = h.getModelStatus()
print("Model Status = ", h.modelStatusToString(model_status))
print("Optimal Objective = ", info.objective_function_value)

Model Status =  Optimal
Optimal Objective =  38714.088107283256


In [8]:
cv = solution.col_value

In [9]:
for i in range(len(cv)):
    if cv[i] > 0.5:
        print(sched.yik[i])

[6, 24, 789.6428975928169, 't0', 0, <scheduler_utils.PossibleStart object at 0x00000250B1348B10>]
[31, 27, 589.112687224596, 't0', 0, <scheduler_utils.PossibleStart object at 0x00000250B13EA5D0>]
[32, 1, 966.564840781936, 't1', 0, <scheduler_utils.PossibleStart object at 0x00000250B13ECC10>]
[33, 13, 188.5491771372604, 't1', 0, <scheduler_utils.PossibleStart object at 0x00000250B13EEA90>]
[36, 93, 67.6505827194146, 't4', 0, <scheduler_utils.PossibleStart object at 0x00000250B140BE90>]
[55, 11, 1522.4043285722641, 't3', 0, <scheduler_utils.PossibleStart object at 0x00000250B1470F10>]
[60, 3, 1742.6443059376518, 't3', 0, <scheduler_utils.PossibleStart object at 0x00000250B1486190>]
[70, 6, 1978.0724539871, 't0', 0, <scheduler_utils.PossibleStart object at 0x00000250B16E5B50>]
[86, 21, 1549.8383807970579, 't3', 0, <scheduler_utils.PossibleStart object at 0x00000250B173B450>]
[104, 75, 332.2713207766198, 't1', 0, <scheduler_utils.PossibleStart object at 0x00000250B179D4D0>]
[113, 40, 984.0

In [10]:
h_scheduled = []
rv = solution.row_value
for i in range(len(rv)):
    if rv[i] > 0.5:
        const_name = h.getRowName(i)[1]
        if "one_per_reqid_constraint_" in const_name:
            h_scheduled.append(const_name.replace("one_per_reqid_constraint_", ""))
print(h_scheduled)

['6', '31', '32', '33', '36', '55', '60', '70', '86', '104', '113', '114', '117', '141', '143', '153', '167', '170', '203', '208', '213', '250', '264', '270', '301', '302', '314', '317', '323', '361', '375', '398', '414', '416', '425', '432', '439', '467', '470']


In [11]:
list(g_sol["scheduled"].keys()) == h_scheduled

True

### Combined

In [12]:
from scheduler_v2 import Scheduler
from ScheduleSimulator import SchedulerSimulation as SchedSim

In [13]:
s = SchedSim("sample_input/sample_input_v2_3.json")
s.process_event(s.events[0])
requests, resources = s.prepare_scheduler()
sched = Scheduler(s.now, s.horizon, s.slice_size, resources, s.proposals, requests, verbose=0)
sched.calculate_free_windows()
sched.build_data_structures()
sched.build_model()

In [14]:
sched.solve_model()

21693


In [15]:
sched.solution

[-0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 

In [16]:
sched.solve_with_highs()

(38714.088107283256,
 {'scheduled': {'6': {'rID': 6,
    'resource': 't0',
    'start': 12900,
    'end': 14081,
    'duration': 1181,
    'priority': 789.6428975928169},
   '31': {'rID': 31,
    'resource': 't0',
    'start': 14100,
    'end': 14988,
    'duration': 888,
    'priority': 589.112687224596},
   '32': {'rID': 32,
    'resource': 't1',
    'start': 300,
    'end': 1756,
    'duration': 1456,
    'priority': 966.564840781936},
   '33': {'rID': 33,
    'resource': 't1',
    'start': 1800,
    'end': 2123,
    'duration': 323,
    'priority': 188.5491771372604},
   '36': {'rID': 36,
    'resource': 't4',
    'start': 14890,
    'end': 15104,
    'duration': 214,
    'priority': 67.6505827194146},
   '55': {'rID': 55,
    'resource': 't3',
    'start': 9900,
    'end': 12193,
    'duration': 2293,
    'priority': 1522.4043285722641},
   '60': {'rID': 60,
    'resource': 't3',
    'start': 2100,
    'end': 4719,
    'duration': 2619,
    'priority': 1742.6443059376518},
   '70'

In [17]:
sched.gurobi_solve_time

0.8298239707946777

In [18]:
sched.highs_solve_time

18.630943059921265

In [19]:
vars(sched.model.getConstrs()[6])

{'_cmodel': <capsule object NULL at 0x00000250B1BAFB70>, '_rowno': 6}

In [20]:
a = sched.model.getJSONSolution()

In [21]:
import json
json.loads(a)

{'SolutionInfo': {'Status': 2,
  'Runtime': 0.8300001621246338,
  'Work': 0.822935926542864,
  'ObjVal': 38714.088107283256,
  'ObjBound': 38728.70566797898,
  'ObjBoundC': 38728.70566797898,
  'MIPGap': 0.00037757729576938813,
  'IntVio': 0,
  'BoundVio': 0,
  'ConstrVio': 0,
  'IterCount': 176,
  'BarIterCount': 23,
  'NodeCount': 1,
  'SolCount': 4,
  'PoolObjBound': 38728.70566797898,
  'PoolObjVal': [38714.088107283256,
   36598.87811435307,
   36511.71920131412,
   22175.989190077886]},
 'Vars': [{'VarName': '6', 'X': 1},
  {'VarName': '31', 'X': 1},
  {'VarName': '32', 'X': 1},
  {'VarName': '33', 'X': 1},
  {'VarName': '36', 'X': 1},
  {'VarName': '55', 'X': 1},
  {'VarName': '60', 'X': 1},
  {'VarName': '70', 'X': 1},
  {'VarName': '86', 'X': 1},
  {'VarName': '104', 'X': 1},
  {'VarName': '113', 'X': 1},
  {'VarName': '114', 'X': 1},
  {'VarName': '117', 'X': 1},
  {'VarName': '141', 'X': 1},
  {'VarName': '143', 'X': 1},
  {'VarName': '153', 'X': 1},
  {'VarName': '167', 'X'

In [22]:
a

'{ "SolutionInfo": { "Status": 2, "Runtime": 8.3000016212463379e-01, "Work": 0.822935926542864, "ObjVal": 3.8714088107283256e+04, "ObjBound": 3.8728705667978982e+04, "ObjBoundC": 3.8728705667978982e+04, "MIPGap": 3.7757729576938813e-04, "IntVio": 0, "BoundVio": 0, "ConstrVio": 0, "IterCount": 176, "BarIterCount": 23, "NodeCount": 1, "SolCount": 4, "PoolObjBound": 3.8728705667978982e+04, "PoolObjVal": [ 3.8714088107283256e+04, 3.6598878114353072e+04, 3.6511719201314118e+04, 2.2175989190077886e+04]}, "Vars": [ { "VarName": "6", "X": 1}, { "VarName": "31", "X": 1}, { "VarName": "32", "X": 1}, { "VarName": "33", "X": 1}, { "VarName": "36", "X": 1}, { "VarName": "55", "X": 1}, { "VarName": "60", "X": 1}, { "VarName": "70", "X": 1}, { "VarName": "86", "X": 1}, { "VarName": "104", "X": 1}, { "VarName": "113", "X": 1}, { "VarName": "114", "X": 1}, { "VarName": "117", "X": 1}, { "VarName": "141", "X": 1}, { "VarName": "143", "X": 1}, { "VarName": "153", "X": 1}, { "VarName": "167", "X": 1}, { "

---

In [4]:
import numpy as np

In [6]:
np.array_split(["t1", "t2", "t3", "t4", "t5", "t6", "t7", "t8"], 2)

[array(['t1', 't2', 't3', 't4'], dtype='<U2'),
 array(['t5', 't6', 't7', 't8'], dtype='<U2')]

In [20]:
from collections import defaultdict
import json
import pprint

In [14]:
a = defaultdict(lambda: defaultdict(lambda: []))

In [16]:
a[1][2]

[]

In [17]:
a

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: []})})

In [18]:
a[1][2] = 3

In [19]:
a

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: 3})})

In [21]:
pprint.pprint(json.loads(json.dumps(a)))

{'1': {'2': 3}}


In [22]:
print(dict(a))

{1: defaultdict(<function <lambda>.<locals>.<lambda> at 0x0000018CF1F85580>, {2: 3})}


In [24]:
for k, v in a.items():
    print(k)
    print(dict(v))

1
{2: 3}


In [25]:
a

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: 3})})

In [26]:
a[1][1]

[]

In [27]:
a[2][2].append(3)

In [28]:
a

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: 3, 1: []}),
             2: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: [3]})})